# Problem set 2

Network models of linear and integer programming.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-8---Network-Models" data-toc-modified-id="Chapter-8---Network-Models-1">Chapter 8 - Network Models</a></span><ul class="toc-item"><li><span><a href="#Problem-7" data-toc-modified-id="Problem-7-1.1">Problem 7</a></span></li><li><span><a href="#Problem-13" data-toc-modified-id="Problem-13-1.2">Problem 13</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-2">Reference</a></span></li></ul></div>

In [1]:
using Cbc, JuMP

## Chapter 8 - Network Models

### Problem 7

An automobile association is organizing a series of car races that will last for four days.
The organizers know that $r_j \ge 0$ special tires in good condition will be required on each of the four successive days, $j = 1, 2, 3, 4$.
They can meet these needs either by buying new tires at $P$ dollars apiece or by reshaping used tires (reshaping is a technique by which the grooves on the tire are deepened, using a special profile-shaped tool).
Two kinds of service are available for reshaping:
normal service, which takes one full day at $N$ dollars a tire, and quick service, which takes overnight at $Q$ dollars a tire.
How should the association, which starts out with no special tires, meet the daily requirements at minimal cost?

**(a)** Formulate a mathematical model for the above problem.
Does it exhibit the characteristics of a network problem?
Why?
(Hint.
Take into account the fact that, at the end of day $j$, some used tires may not be sent to reshaping.)

**Solution:**

The formulation as a network model is as follows.

<img src="../images/mit_problem_7.png" alt="Drawing" style="width: 600px;"/>

There are a node for the tires store, one node for each day and also one node for each day stock.

The stock can send tires to each day, with no restriction in number of tires sold and also no limits to transportation to it day, but each tire costs $P$.

All used tires in one day must be sent to day stock, so the number of tires in node day $i$ is zero. Each stock must send all tires to another stock or another day (through normal or quick services), so the number of tires in each stock is zero (except for last day stock, with must have $\ge 0$ tires).

The transportation among stocks has no limits in number and it is free. The reshaping from each stock to a specific day is unlimited, mas costs $Q$ for quick service, and $N$ for normal service.

**Aditional constraints are necessary**: The number of tires leaving day $i$ must be greater than or equal to $r_i$. This aditional constraint grants that each day will have its tires. So we add a minimum flow to the transportation at the end of each day.

In [2]:
# A reasonable case is when P > Q > N. For example, if P < Q, it wouldn't make sense to use Q at all.

P = 10
N = 5
Q = 8
r = [2, 2, 2, 2]

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of tires purchased
@variable(m, p[1:4] >= 0, Int)

# Flow of tires to stock
@variable(m, s[1:4] >= 0, Int)

# Flow among stocks
@variable(m, a[1:3] >= 0, Int)

# Quick services
@variable(m, q[1:3] >= 0, Int)

# Normal services
@variable(m, n[1:3] >= 0, Int)

# Maximize the costs
@objective(m, Min, P*sum(p) + Q*sum(q) + N*sum(n))

# Constraints through each day
@constraint(m, p[1] - s[1] == 0)
@constraint(m, p[2] + q[1] - s[2] == 0)
@constraint(m, p[3] + n[1] + q[2] - s[3] == 0)
@constraint(m, p[4] + n[2] + q[3] - s[4] == 0)

# Constraints through each stock
@constraint(m, s[1] - q[1] - n[1] - a[1] == 0)
@constraint(m, s[2] + a[1] - q[2] - n[2] - a[2] == 0)
@constraint(m, s[3] + a[2] - q[3] - a[3] == 0)
@constraint(m, s[4] + a[3] >= 0)

# Constraints of flow from each day
@constraint(m, s .>= r)

optimize!(m)

In [3]:
println("Cost: ", objective_value(m))
for i=1:4
    println("p_$i qty: ", value(p[i]))
end
for i=1:3
    println("q_$i qty: ", value(q[i]))
end
for i=1:2
    println("n_$i qty: ", value(n[i]))
end

Cost: 60.0
p_1 qty: 2.0
p_2 qty: 2.0
p_3 qty: 0.0
p_4 qty: 0.0
q_1 qty: 0.0
q_2 qty: 0.0
q_3 qty: 0.0
n_1 qty: 2.0
n_2 qty: 2.0


The program is very smart. We could think that, because $Q < P$, we would have only the first tires purchased, and then have the rest of the tires reshaped.
But it would require to use $Q$, however $N$ is cheaper.
So much cheaper that it is worth it to buy more tires to use only $N$.

**(b)** If the answer to **(a)** is no, how can the formulation be manipulated to become a network problem?
Draw the associated network.
(Hint.
Add a redundant constraint introducing a fictitious node.)

**Solution:** The answer was yes.

**(c)** Assume that a tire may be reshaped only once.
How does the above model change?
Will it still be network problem?

**Solution:** We will continue with the image above.
The flow through each day will only consider the black arrows, that is, reshaped tires will not go to stock.
But we will require that the total of purchased and reshaped tires will have to met the requirements of each day.

In [4]:
# A reasonable case is when P > Q > N. For example, if P < Q, it wouldn't make sense to use Q at all.

P = 10
N = 5
Q = 6
r = [2, 2, 2, 2]

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of tires purchased
@variable(m, p[1:4] >= 0, Int)

# Flow of tires to stock
@variable(m, s[1:4] >= 0, Int)

# Flow among stocks
@variable(m, a[1:3] >= 0, Int)

# Quick services
@variable(m, q[1:3] >= 0, Int)

# Normal services
@variable(m, n[1:3] >= 0, Int)

# Minimize the costs
@objective(m, Min, P*sum(p) + Q*sum(q) + N*sum(n))

# Constraints through each day
# We do not consider the reshaped tires flowing from the day to the stock again
@constraint(m, p[1] - s[1] == 0)
@constraint(m, p[2] - s[2] == 0)
@constraint(m, p[3] - s[3] == 0)
@constraint(m, p[4] - s[4] == 0)

# Constraints through each stock
@constraint(m, s[1] - q[1] - n[1] - a[1] == 0)
@constraint(m, s[2] + a[1] - q[2] - n[2] - a[2] == 0)
@constraint(m, s[3] + a[2] - q[3] - a[3] == 0)
@constraint(m, s[4] + a[3] >= 0)

# Constraints of flow entering each day
@constraint(m, p[1] >= r[1])
@constraint(m, p[2] + q[1] >= r[2])
@constraint(m, p[3] + q[2] + n[1] >= r[3])
@constraint(m, p[4] + q[3] + n[2] >= r[4])

optimize!(m)

In [5]:
println("Cost: ", objective_value(m))
for i=1:4
    println("p_$i qty: ", value(p[i]))
end
for i=1:3
    println("q_$i qty: ", value(q[i]))
end
for i=1:2
    println("n_$i qty: ", value(n[i]))
end

Cost: 60.0
p_1 qty: 2.0
p_2 qty: 2.0
p_3 qty: 0.0
p_4 qty: 0.0
q_1 qty: 0.0
q_2 qty: 0.0
q_3 qty: 0.0
n_1 qty: 2.0
n_2 qty: 2.0


The solution was maintened, because that solution was already reshaping tires at most once.
However, it is not a network model anymore, because we are not considering the balance of the nodes of the days as it is required to be: 

$$\textrm{(Flow out of the node)} -  \textrm{(Flow into the node)} = \textrm{(Net supply of the node)}$$

### Problem 13

A manufacturer of small electronic calculators is working on setting up his production plans for the next six months.
One product is particularly puzzling to him.
The orders on hand for the coming season are:

| Month | Orders |
| --- | --- |
| January | 100 |
| February | 150 |
| March | 200 |
| April | 100 |
| May | 200 |
| June | 150 |

The product will be discontinued after satisfying the June demand.
Therefore, there is no need to keep any inventory after June.
The production cost, using regular manpower, is \\$10 per unit.
Producing the calculator on overtime costs an additional \\$2 per unit.
The inventory-carrying cost is \\$0.50 per unit per month.
If the regular shift production is limited to 100 units per month and overtime production is limited to an additional 75 units per month, what is the optimal production schedule?
(*Hint.* Treat regular and overtime capacities as sources of supply for each month.)

**Solution:** The following network represents the network model:

<img src="../images/mit_problem_13.png" alt="Drawing" style="width: 400px;"/>

The normal and overtime nodes do not have constraints in their capacity.
Blue arrows have a limit of 100 units, and a cost of \\$10 per unit.
Red arrows have limit of 75 units, and a cost of \\$12 per unit.
Black arrows do not have constraint in the number of units, and the cost per unit is \\$0.50.
The difference between what enters and what leaves a node must be equal to (or even greater than) the order, for each month.

Let's solve it:

In [10]:
O = [100, 150, 200, 100, 200, 150]

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Normal operation
@variable(m, 0 <= n[1:6] <= 100, Int)

# Overtime operation
@variable(m, 0 <= o[1:6] <= 75, Int)

# Inventory-carrying
# The first and the last is forced to be zero
@variable(m, 0 <= i[1:7], Int)
@constraint(m, i[1] == 0)
@constraint(m, i[7] == 0)

# Minimize the costs
@objective(m, Min, 10*sum(n) + 12*sum(o) + 0.5*sum(i))

# Constraints through each month
for j in 1:6
    @constraint(m, n[j] + o[j] + i[j] - i[j+1] >= O[j])
end

optimize!(m)

In [14]:
println("Cost: ", objective_value(m))
println()
for j=1:6
    println("n_$j qty: ", value(n[j]))
end
println()
for j=1:6
    println("o_$j qty: ", value(o[j]))
end
println()
for j=2:6
    println("i_$j qty: ", value(i[j]))
end

Cost: 9625.0

n_1 qty: 100.0
n_2 qty: 100.0
n_3 qty: 100.0
n_4 qty: 100.0
n_5 qty: 100.0
n_6 qty: 100.0

o_1 qty: 0.0
o_2 qty: 75.0
o_3 qty: 75.0
o_4 qty: 25.0
o_5 qty: 75.0
o_6 qty: 50.0

i_2 qty: 0.0
i_3 qty: 25.0
i_4 qty: 0.0
i_5 qty: 25.0
i_6 qty: 0.0


## Reference

```bibtex
@book{bradley1977applied,
  title={Applied mathematical programming},
  author={Bradley, Stephen P and Hax, Arnoldo C and Magnanti, Thomas L},
  year={1977},
  publisher={Addison-Wesley}
}
```